In [ ]:
from uqdd.models.utils_models import load_model, get_model_config
from uqdd.models.baseline import BaselineDNN
from uqdd.models.evidential import EvidentialDNN
from uqdd.models.mcdropout import run_mcdropout # we will use baseline with run_mcdp

# METRICS
from uqdd.models.utils_train import evaluate_predictions
# from uqdd.utils import get_config
from uqdd.utils import load_df
from uqdd.models.utils_metrics import MetricsTable


In [ ]:
# Load csv runs and model names file
runs_path = 'runs.csv'
runs_df = load_df(runs_path)

# Load predictions for each model_name
# predictions_path = 'predictions.csv'
preds_dirpath = '/home/bkhalil/Repos/uqdd/uqdd/data/predictions/papyrus/xc50/all/'
filename = 'preds.csv'
predictions_df = load_df(preds_dirpath+filename)


In [1]:

# get arrays from df cols
y_true = predictions_df['y_true'].values
y_pred = predictions_df['y_pred'].values
y_err = predictions_df['y_err'].values
y_alea = predictions_df['y_alea'].values
eps_col = 'y_std' if 'y_std' in predictions_df.columns else 'y_eps'
y_eps = predictions_df[eps_col].values

# we need to get model_type and config from runs_df
model_type = "ensemble"

config = get_model_config(
    model_type,
    data_name="papyrus",
    activity_type="xc50",
    split_type="random",
    descriptor_protein="ankh-large",
    descriptor_chemical="ecfp2048",
    data_specific_path="papyrus/xc50/all"
)


uct_metrics_logger = MetricsTable(
    model_type=model_type,
    config=config,
    add_plots_to_table=False,
    # * we can turn on if we want to see them in wandb * #
    # logger=logger,
    project_name=None, # to change as this will become the output csv file name
    run_name=None, # that would be the same run name from the runs.csv file
)
task_name = "PCM"
metrics, plots = uct_metrics_logger(
        y_pred=y_pred,
        y_std=y_alea,
        y_true=y_true,
        y_err=y_err,
        # y_alea=y_alea,
        y_eps=y_eps,
        task_name=task_name,
        figpath=None, # Here we need to define the fig path 
        # the figpath default was FIGS_DIR / data_specific_path / self.model_name
        # we can change it to mark the reassessment of the model somehoe (maybe add a prefix to model_name like reasses)
        # FIGS_DIR is imported variable from uqdd
        # data_specific_path is predefined var data_specific_path="papyrus/xc50/all"
    )

# * calculate metrics for a subset of the datapoints * #
submetrics, subplots = uct_metrics_logger(
    y_pred=y_pred,
    y_std=y_alea,
    y_true=y_true,
    y_err=y_err,
    # y_alea=y_alea,
    y_eps=y_eps,
    task_name=task_name+" Subset",
    n_subset=100,
)

# then we log to the csv file which appends after each iteration
uct_metrics_logger.csv_log()


NameError: name 'predictions_df' is not defined